In [16]:
import os
import glob

import cv2
import cvlib as cv
import numpy as np
import pandas as pd
from random import shuffle
from keras.preprocessing.image import ImageDataGenerator

from Tgram import Tgram
CROP_SIZE = (96,96)


In [17]:
class Datapreprocess:
    def __init__(self):
        self.data_lst = []
        self.img = None
        self.X_position = (0, 0)
        self.Y_position = (0, 0)
        self.curr_emotion = ''
        self.new_file_path = ""

    def _save_crop_img(self):
        try:
            img = self.img.copy()
            roi = img[
                self.Y_position[0] : self.Y_position[1],
                self.X_position[0] : self.X_position[1],
            ]
            img = cv2.resize(roi, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.img = img
            return True
        except:
            return False

    def _detect_face(self, img_path):
        try:
            self.img = cv2.imread(img_path)
            faces, _ = cv.detect_face(self.img, enable_gpu=True)
            self.X_position = faces[0][0], faces[0][2]
            self.Y_position = faces[0][1], faces[0][3]
            return True
        except:
            return False

    def _random_name(self):
        rand_int = list(map(str,np.random.randint(low=0, high=9, size=7)))
        lst = list(map(chr, np.random.randint(low=97, high=122, size=45)))+rand_int
        shuffle(lst)
        return "".join(lst)


    def work(self, img_path,emo):
        self.curr_emotion = emo
        if self._detect_face(img_path) and self._save_crop_img():
            if self.img.shape == CROP_SIZE:
                self.new_file_path = f"./dataset/{self.curr_emotion}_{self._random_name()}.jpg"
                self.data_lst.append({"path": self.new_file_path, "label": self.curr_emotion})
                cv2.imwrite(self.new_file_path, self.img)
                return True
            else:
                return False

In [18]:
tgram = Tgram()
datapreprocess = Datapreprocess()
path_dir_lst = os.listdir("./pre_dataset/")


print(path_dir_lst)
folder_lst = [
    f'./pre_dataset/{e}/'
    for e in path_dir_lst
]
print(folder_lst)
for emo,folder in zip(path_dir_lst,folder_lst):
    try:
        img_list = glob.glob(os.path.join(folder)+"*.jpg")
        img_list = list(map(lambda x : x.replace("\\",'/'),img_list))
        for img_path in img_list:
            datapreprocess.work(img_path,emo)
    except Exception as e:
        tgram.fail(e)
pd.DataFrame(datapreprocess.data_lst).to_csv("dataset.csv", index=False)
tgram.success()

[]
[]
